<center><img src="images/logo_CESI.png" width="250"></center>

# <center><b>Projet algorithmique et optimisation combinatoire</b></center>
# <center><b>Livrable 1</b></center>

<p><center><b><u>Groupe de projet</u> :</b></center></p>
<center><b>Mickaël King</b></center>
<center><b>(À compléter)</b></center>
<center><b>Zie Mohamed Lamine Traore</b></center>
<center><b>(À compléter)</b></center>

<p><center><b><u>Tuteur</u> : Mohamed Belgacem</b></center></p>
<center><b><u>Promotion</u> : FISE A3 Informatique</b></center>

# Sommaire

1. [Contexte](#Contexte)
2. [Problématique](#Problematique)
3. [Propriété théorique](#Propriete-theorique)
4. [Contraintes](#Contraintes)
5. [Représentation des données](#Representation-des-donnees)
    * [Modélisation sous forme de graphe](#Modelisation-sous-forme-de-graphe)
    * [Modélisation des camions](#Modelisation-des-camions)
6. [Complexité du problème](#Complexite-du-probleme)
7. [Solutions et outils proposées](#Solutions-et-outils-proposees)
8. [Bibliographie](#Bibliographie)

# Contexte

Ce projet s’inscrit dans le cadre du module d’algorithmique et d’optimisation combinatoire.
Il vise à appliquer les méthodes vues en cours à un problème concret d’optimisation, notamment la planification des tournées de véhicules pour la livraison de marchandises.
L’objectif général est de modéliser un problème réel de transport sous forme mathématique et algorithmique, puis de proposer une solution heuristique permettant de trouver des trajets efficaces tout en respectant différentes contraintes.

# Problematique

(À compléter -> Ryan)

# Propriete theorique

(À compléter -> Oumar)

# Contraintes

Le modèle que nous avons développé doit prendre en compte plusieurs contraintes réelles et combinatoires, issues des spécifications du VRP et de ses variantes.

1️⃣ Contraintes de capacité

Chaque véhicule possède une capacité maximale qu’il ne doit pas dépasser.
Pour chaque route 𝑟, la somme des demandes 𝑑𝑖 des clients visités doit être inférieure ou égale à la capacité du véhicule :
$$
\sum_{i \in R} d_i \le Q
$$


2️⃣ Contraintes de fenêtres temporelles

Chaque client 𝑖 est associé à une fenêtre temporelle [𝑎𝑖,𝑏𝑖] pendant laquelle la livraison doit avoir lieu.Le véhicule peut arriver avant 𝑎𝑖(il attend), mais pas après 𝑏𝑖, sinon la solution est jugée infaisable ou pénalisée.

3️⃣ Contraintes de flotte hétérogène

Les véhicules disponibles ne sont pas identiques :
certains ont une capacité plus élevée mais un coût d’exploitation supérieur ;d’autres ne sont compatibles qu’avec certains types de marchandises (ex. camions réfrigérés, camions « peluches », etc.).Chaque véhicule est donc caractérisé par un triplet :
(capacite,cout,type_de_marchandise)

4️⃣ Contraintes de trajet et de tournée

Chaque client est visité exactement une fois.Chaque tournée commence et se termine au dépôt.Le véhicule ne peut pas dépasser sa capacité ni violer les contraintes temporelles.

5️⃣ Contraintes de faisabilité inter-dépôts

Dans certains cas, les dépôts sont spécialisés : un dépôt ne peut réapprovisionner qu’un type spécifique de marchandise.
Ainsi, la compatibilité entre les camions et les dépôts fait également partie du modèle.

# Representation des donnees

Pour modéliser le problème de tournées de véhicules, il est nécessaire de définir précisément les ensembles, paramètres et variables utilisés.  

### 🔹 1. Ensembles
- \( V = \{0, 1, 2, ..., n\} \) : ensemble des sommets (le nœud \( 0 \) représente le dépôt).  
- \( C = V \setminus \{0\} \) : ensemble des clients à livrer.  
- \( K = \{1, 2, ..., m\} \) : ensemble des véhicules disponibles.  

### 🔹 2. Paramètres
- \( d_i \) : demande (poids, volume ou nombre d’unités) du client \( i \).  
- \( Q_k \) : capacité maximale du véhicule \( k \).  
- \( t_{ij} \) : temps ou coût de trajet entre les points \( i \) et \( j \).  
- \( [a_i, b_i] \) : fenêtre temporelle de livraison autorisée pour le client \( i \).  
- \( s_i \) : temps de service (durée de la livraison) pour le client \( i \).  
- \( \text{type}(k) \) : type du véhicule \( k \) (dans le cas d’une flotte hétérogène).  

### 🔹 3. Variables de décision
- \( x_{ijk} = 1 \) si le véhicule \( k \) emprunte l’arc \( (i,j) \), et \( 0 \) sinon.  
- \( t_i \) : heure d’arrivée du véhicule au client \( i \).  

### 🔹 4. Données d’entrée
Les données peuvent être représentées sous la forme :
- d’une **matrice de distance ou de temps** \( T = [t_{ij}] \),  
- d’un **vecteur de demandes** \( D = [d_1, d_2, ..., d_n] \),  
- d’un **tableau de véhicules** avec leurs capacités et types,  
- et d’un **ensemble de fenêtres temporelles** pour chaque client.

### 🔹 5. Exemple de représentation
Exemple simplifié sous forme de dictionnaires Python :  
```python
# Exemple de structure de données
clients = {
    1: {"demand": 10, "time_window": (8, 12)},
    2: {"demand": 15, "time_window": (9, 14)},
    3: {"demand": 8,  "time_window": (10, 16)}
}

vehicles = {
    1: {"capacity": 25, "type": "électrique"},
    2: {"capacity": 40, "type": "diesel"}
}

# Matrice des distances (symétrique)
distance_matrix = [
    [0, 10, 20, 15],
    [10, 0, 25, 30],
    [20, 25, 0, 35],
    [15, 30, 35, 0]
]




### Modelisation sous forme de graphe

D'un point de vue <b>théorique</b>, notre réseau de transport sera modéliser sous forme de <b>graphe</b>.

Nous partons du principe que chaque <b>villes</b> et représenter par un <b><u>sommet</u></b> et que chaque <b>liaison</b> entre ces villes seront représenter par une <b><u>arrête</u></b>.

<center><img src="images/representation_exemple.png" width="250"></center>

Dans l'exemple ci-dessus, nous avons 3 villes <b>A</b>, <b>B</b> et <b>C</b>.
En supposant qu'il n'y ait pas de sens unique, on peut passer de la ville <b>A</b> à la ville et de la ville <b>B</b> à la ville <b>C</b> et vice-versa.

Notre graphe pour modéliser le contexte et donc un <b><u>graphe non orienté</u></b>.

Quant à la contrainte des <b>points de collectes</b>, nous avons décidé d'attribuer pour chaque point de collecte spécifique à une ville différente. C'est à dire qu'un sommet représentant une ville peut aussi être considérer comme étant un point de collecte.

<center><img src="images/representation_exemple1.png" width="250"></center>

Par exemple dans l'exemple ci-dessus, le point de collecte se situe à la ville <b>C</b>.

On prendra en compte aussi que certain dépôt ne peut réapprovisionner que <b>certains type d'objets</b>. 

Par exemple si un camion qui livre seulement des peluches tombe à court de peluches à livrer, il ne pourra se réapprovisionner que dans un dépôt qui réapprovisionne des peluches.

### Modelisation des camions

Pour la contrainte de <b>flotte hétérogène</b>, nous imaginons que chaque camions sera stocker en mémoire dans une structure de donnée spécifique qui seront des <b>tableaux</b>.

Sachant que chaque camion ne peut livrer qu'un <b>seul type d'objet</b>, chaque tableaux devra contenir des camions pour un seul type d'objet spécifique uniquement.

Cela nous permettra à la fois <b>d'organiser</b> facilement les camions en fonction du type d'objet mais aussi de pouvoir aller <b>chercher</b> un camion spécifique en particulier en accedant tout simplement au tableau.

<center><img src="images/representation_exemple2.png" width="250"></center>

Dans l'exemple ci-dessus, nous imaginons que des camions qui ne transporte que des peluches seront tous ranger dans un même tableau en commun. Nous avons dans l'exemple 6 camions (A1, A2, ..., A6).

# Complexite du probleme

(À compléter -> Oumar + les autres)

# Solutions et outils proposees

(À compléter -> Ryan)

# Bibliographie

(À compléter -> Tout le monde)

Site qui explique comment implémenter des graphes et appliquer des algorithmes connues d'un point de vue informatique :
https://www.geeksforgeeks.org/dsa/graph-data-structure-and-algorithms/